## Đồ án 02: Logic

- Họ tên: Nguyễn Huỳnh Mẫn
- MSSV: 20126041

<font color='red'>**LƯU Ý:**</font>

- Bài làm của sinh viên phải do chính bản thân sinh viên tự làm, có thể trao đổi và tham khảo ý tưởng nhưng không được sao chép (một phần hoặc toàn bộ) code hoặc lời giải từ bất cứ người nào khác. Nếu vi phạm sẽ bị <font color='red'>0đ</font> bài tập này.
- Các bạn có thể tạo thêm các cell trong quá trình code, tuy nhiên các bạn vui lòng <font color='red'>không xóa các cell code mặc định và các cell test case</font> (vì có thể ảnh hưởng đến kết quả khi chấm bài).
- Các test case được đưa ra chỉ nhằm mục đích giúp các bạn test code của mình, việc pass các test case này không đồng nghĩa với việc lời giải của các bạn sẽ đạt điểm tối đa. 

**Cách nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart Kernel & Run All Cells`, để restart và chạy tất cả các cell trong notebook của các bạn; do đó, trước khi nộp bài, các bạn nên chạy thử `Kernel` - `Restart Kernel & Run All Cells` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, các bạn tạo thư mục nộp bài theo cấu trúc sau:

- Thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - Thư mục `Project02_logic`
        - File `MSSV.ipynb` (Đổi tên file notebook này theo MSSV của bạn)
        - Thư mục chứa bài làm câu 4 đặt tên là `PS4`, gồm:
            - `SRC`: thư mục chứa mã nguồn
            - `REPORT`: thư mục chứa báo cáo (file `*.pdf`)

Cuối cùng, các bạn nén thư mục `MSSV` này lại và nộp ở link trên moodle. **Đuôi của file nén phải là .zip (chứ không được .rar hay gì khác).**

<font color=red>Các bạn lưu ý tuân thủ chính xác qui định nộp bài ở trên.</font>

## Hướng dẫn làm bài: 

Đồ án này hướng đến việc giúp các bạn thực hành và kiểm tra kiến thức chương Logic trong môn học Nhập môn Trí tuệ Nhân tạo. 

Nội dung của đồ án này bao gồm:

- Logic mệnh đề và hợp giải trên logic mệnh đề 
- Logic vị từ 

Với các câu hỏi 1-3, các bạn sẽ làm bài trực tiếp trên notebook này. Các bạn cần hoàn thành notebook bằng cách điền lời giải của mình sau các dòng ```# YOUR CODE HERE```. Các bạn sẽ cần sử dụng code để biểu diễn các ký hiệu trong logic theo cú pháp trong bảng sau: 

<center>
    
|  | Biểu diễn bằng ký hiệu toán học | Cú pháp code sử dụng | 
|:---|:----:|---|
| Ký hiệu hằng số (constant symbol) | hcmus | ```Constant('hcmus')``` (ghi thường) | 
| Ký hiệu biến (variable symbol) | $x$ | ```Variable('$x')``` (ghi thường) | 
| Câu nguyên tố (atomic formula) | Rain, LocatedIn(hcmus, $x$) | ```Atom('Rain')```, ```Atom('LocatedIn', 'hcmus', '$x')``` <br> (Kí hiệu vị từ ghi hoa, đối số ghi thường) |
| Phép phủ định (negation) | $\neg$ Rain | ```Not(Atom('Rain'))```|
| Phép và (conjunction)| Rain $\land$ Snow | ```And(Atom('Rain'), Atom('Snow'))```|
| Phép hoặc (disjunction) | Rain $\lor$ Snow | ```Or(Atom('Rain'), Atom('Snow'))```|
| Phép kéo theo (implication) | Rain $\Rightarrow$ Wet | ```Implies(Atom('Rain'), Atom('Wet'))```|
| Phép tương đương (equivalence) | Rain $\Leftrightarrow$ Wet | ```Equiv(Atom('Rain'), Atom('Wet'))```|
| Lượng từ tồn tại (existential quantification) | $\exists x$ LocatedIn(hcmus,$x$) | ```Exists('$x', Atom('LocatedIn', 'hcmus', '$x'))```|
| Lượng từ với mọi (universal quantification) | $\forall x$ MadeOfAtoms($x$) | ```Forall('$x', Atom('MadeOfAtoms', '$x'))```|    
| Phép và với nhiều đối số <br>(tương tự với phép hoặc)| A $\land$ B $\land$ C | ```AndList([Atom('A'), Atom('B'), Atom('C')]) ```|
  
    
</center>

In [245]:
# util function for testing your code, DO NOT modify!!!

import pickle, gzip, os, random
import sys
import os
import collections

from logic import *
from typing import Tuple, List

def checkFormula(name, predForm, preconditionForm=None):
    filename = os.path.join('models', name + '.pklz')
    objects, targetModels = pickle.load(gzip.open(filename)) 
    
    # If preconditionion exists, change the formula to
    preconditionPredForm = And(preconditionForm, predForm) if preconditionForm else predForm
    predModels = performModelChecking([preconditionPredForm], findAll=True, objects=objects)
    ok = True

    def hashkey(model): return tuple(sorted(str(atom) for atom in model))
    targetModelSet = set(hashkey(model) for model in targetModels)
    predModelSet = set(hashkey(model) for model in predModels)
    for model in targetModels:
        if hashkey(model) not in predModelSet:
            print("Your formula (%s) says the following model is FALSE, but it should be TRUE:" % predForm)
            ok = False
            printModel(model)
            return
    for model in predModels:
        if hashkey(model) not in targetModelSet:
            print("Your formula (%s) says the following model is TRUE, but it should be FALSE:" % predForm)
            ok = False
            printModel(model)
            return
        
    print(f"You matched the {len(targetModels)} models")
    print(f"Example model: {rstr(random.choice(targetModels))}")
    return True


In [246]:
# import other modules if you need them 


## Câu 1: Logic mệnh đề (0.75đ)

Chuyển các câu dưới đây từ ngôn ngữ tự nhiên thành công thức logic mệnh đề.

Ví dụ, với câu: "Nếu trời mưa thì đường trơn" thì cách viết sẽ là:

```
def rainWet():
    Rain = Atom('Rain') # Trời mưa
    Wet = Atom('Wet') # Đường trơn
    return Implies(Rain, Wet)
```


**1a) Nếu hiện tại là mùa hè và chúng tôi đang ở California, thì trời không mưa**

In [247]:
def formula1a() -> Formula:
    # Predicates to use:
    Summer = Atom('Summer')               # whether it's summer
    California = Atom('California')       # whether we're in California
    Rain = Atom('Rain')                   # whether it's raining
    
    # YOUR CODE HERE
    return Implies(And(Summer, California), Not(Rain))
    raise NotImplementedError()

In [248]:
assert checkFormula('1a', formula1a()) == True

You matched the 7 models
Example model: {'Summer'}


**1b) Đường trơn khi và chỉ khi trời mưa hoặc vòi phun nước đang mở**

In [249]:
def formula1b() -> Formula:
    # Predicates to use:
    Rain = Atom('Rain')              # whether it is raining
    Wet = Atom('Wet')                # whether it it wet
    Sprinklers = Atom('Sprinklers')  # whether the sprinklers are on
    # YOUR CODE HERE
    return Equiv(Wet, Or(Rain, Sprinklers))
    raise NotImplementedError()

In [250]:
assert checkFormula('1b', formula1b()) == True

You matched the 4 models
Example model: {'Wet', 'Sprinklers'}


**1c) Hoặc là ngày hoặc là đêm (nhưng không đồng thời là hai trường hợp trên)**

In [251]:
def formula1c() -> Formula:
    # Predicates to use:
    Day = Atom('Day')     # whether it's day
    Night = Atom('Night') # whether it's night
    # YOUR CODE HERE
    return Or(And(Day, Not(Night)), And(Not(Day), Night))   
    # A⊕B≡(A∧¬B)∨(¬A∧B) 
    raise NotImplementedError()

In [252]:
assert checkFormula('1c', formula1c()) == True

You matched the 2 models
Example model: {'Day'}


## Câu 2: Logic vị từ (2đ)

Chuyển các câu dưới đây từ ngôn ngữ tự nhiên thành công thức logic vị từ.

Ví dụ, với câu: "Tồn tại một nguồn sáng phát sáng"

```
def lightShines():
    def Light(x): return Atom('Light', x)    # x là nguồn sáng
    def Shines(x): return Atom('Shines', x)  # x phát sáng
    return Exists('$x', And(Light('$x'), Shines('$x')))
```

**2a) Mọi người đều có mẹ**

In [253]:
def formula2a() -> Formula:
    # Predicates to use:
    def Person(x): return Atom('Person', x)        # whether x is a person
    def Mother(x, y): return Atom('Mother', x, y)  # whether x's mother is y

    # Note: You do NOT have to enforce that the mother is a "person"
    # YOUR CODE HERE
    return Forall('$x', Implies(Person('$x'),Exists('$y', Mother('$x', '$y'))))
    raise NotImplementedError()

In [254]:
formula2a_precondition = AntiReflexive('Mother')
assert checkFormula('2a', formula2a(), formula2a_precondition) == True

You matched the 343 models
Example model: {'Mother(o3,o1)', 'Mother(o2,o1)', 'Mother(o1,o3)', 'Mother(o3,o2)'}


**2b) Ít nhất một người không có con cái**

In [255]:
def formula2b() -> Formula:
    # Predicates to use:
    def Person(x): return Atom('Person', x)        # whether x is a person
    def Child(x, y): return Atom('Child', x, y)    # whether x has a child y

    # Note: You do NOT have to enforce that the child is a "person"
    # YOUR CODE HERE
    return Exists('$x', And(Person('$x'), Not(Exists('$y', Child('$x', '$y')))))
    raise NotImplementedError()

In [256]:
formula2b_precondition = AntiReflexive('Child')
assert checkFormula('2b', formula2b(), formula2b_precondition) == True

You matched the 169 models
Example model: {'Child(o2,o1)', 'Person(o3)', 'Child(o2,o3)'}


**2c) Viết công thức định nghĩa Daughter (con gái) theo Female (nữ) và Child (con)**

Ví dụ về dạng 2c và 2d:

```
# Định nghĩa Parent theo Child
def parentChild():
    def Parent(x, y): return Atom('Parent', x, y)  # whether x has a parent y
    def Child(x, y): return Atom('Child', x, y)    # whether x has a child y
    return Forall('$x', Forall('$y', Equiv(Parent('$x', '$y'), Child('$y', '$x'))))
    
```

In [257]:
def formula2c() -> Formula:
    # Predicates to use:
    def Female(x): return Atom('Female', x)            # whether x is female
    def Child(x, y): return Atom('Child', x, y)        # whether x has a child y
    def Daughter(x, y): return Atom('Daughter', x, y)  # whether x has a daughter y
    
    # YOUR CODE HERE
    return Forall('$x', Forall('$y', Equiv(Daughter('$x', '$y'), And(Child('$x', '$y'), Female('$y')))))
    raise NotImplementedError()

In [258]:
formula2c_precondition = AntiReflexive('Child')
assert checkFormula('2c', formula2c(), formula2c_precondition) == True

You matched the 512 models
Example model: {'Child(o2,o1)', 'Female(o3)', 'Daughter(o2,o3)', 'Child(o3,o2)', 'Child(o2,o3)'}


**2d) Viết công thức định nghĩa Grandmother (bà) theo Female (nữ) và Parent (phụ huynh)**

In [259]:
# Note: It is ok for a person to be her own parent
def formula2d() -> Formula:
    # Predicates to use:
    def Female(x): return Atom('Female', x)                  # whether x is female
    def Parent(x, y): return Atom('Parent', x, y)            # whether x has a parent y
    def Grandmother(x, y): return Atom('Grandmother', x, y)  # whether x has a grandmother y
    # YOUR CODE HERE
    return Forall('$x', Forall('$y', Equiv(Grandmother('$y', '$x'), 
                                           Exists('$z', 
                                                  And(
                                                      Parent('$y', '$z'), 
                                                      And(Parent('$z', '$x'), Female('$x')))))))
    raise NotImplementedError()

In [260]:
formula2d_precondition = AntiReflexive('Parent')
assert checkFormula('2d', formula2d(), formula2d_precondition) == True

You matched the 512 models
Example model: {'Parent(o2,o3)', 'Grandmother(o2,o1)', 'Female(o2)', 'Grandmother(o3,o3)', 'Parent(o3,o2)', 'Grandmother(o3,o2)', 'Parent(o1,o2)', 'Female(o3)', 'Female(o1)', 'Grandmother(o2,o2)', 'Grandmother(o1,o3)', 'Parent(o3,o1)'}


## Câu 3: Logic vị từ - Ai là người nói dối? (2.25đ)

Trong bài tập này, ta sẽ vận dụng logic vị từ để giải bài toán sau đây:

Một nhân viên nào đó ở Facebook đã gây ra lỗi nghiêm trọng làm sập server, và mọi người đang bàn tán xem ai là người gây ra sự cố này. Sau một hồi điều tra, người ta đã xác định được 4 nghi phạm: John, Susan, Mark, Nicole. 

Ta có các thông tin sau:

- John khai rằng: "Tôi không phải là người làm sập server!"
- Susan nói: "Thủ phạm chính là Nicole"
- Mark nói: "Không đâu, Susan làm sập server đó!"
- Nicole khai rằng: "Susan nói dối đó, mọi người đừng tin"
- Chỉ có **duy nhất 1 người** nói thật 
- Chỉ có **duy nhất 1 người** làm sập server 

Các bạn hãy hoàn thành hàm ```liar()``` trả về một danh sách chứa 6 công thức logic vị từ ứng với 6 thông tin đã cho để tìm ra người đã nói dối trong quá trình điều tra nhé!

In [261]:
# Problem 3: Liar puzzle

# Facts:
# 0. John: "It wasn't me!"
# 1. Susan: "It was Nicole!"
# 2. Mark: "No, it was Susan!"
# 3. Nicole: "Susan's a liar."
# 4. Exactly one person is telling the truth.
# 5. Exactly one person crashed the server.
# Query: Who did it?

# This function returns a list of 6 formulas corresponding to each of the
# above facts. Be sure your formulas are exactly in the order specified. 


# Hint: For fact 4 & 5, you might want to use the Equals predicate, defined as:
# def Equals(x, y): return Atom('Equals', x, y)
# This predicate is used to assert that two objects are the same.
# In particular, Equals(x,x) = True and Equals(x,y) = False iff x is not equal to y. 
# It can also be solved in other ways, without the Equals predicate!
    
def liar() -> Tuple[List[Formula], Formula]:
    def TellTruth(x): return Atom('TellTruth', x)
    def CrashedServer(x): return Atom('CrashedServer', x)
    john = Constant('john')
    susan = Constant('susan')
    nicole = Constant('nicole')
    mark = Constant('mark')
    formulas = []
    # We provide the formula for fact 0 here.
    formulas.append(Equiv(TellTruth(john), Not(CrashedServer(john))))
    
    # You should add 5 formulas, one for each of facts 1-5.
    # YOUR CODE HERE
     # Fact 1: Susan says it was Nicole
    formulas.append(Equiv(TellTruth(susan), CrashedServer(nicole)))

    # Fact 2: Mark says it was Susan
    formulas.append(Equiv(TellTruth(mark), CrashedServer(susan)))

    # Fact 3: Nicole says Susan's a liar
    formulas.append(Equiv(TellTruth(nicole), Not(TellTruth(susan))))

    
    # Fact 4: Exactly one person is telling the truth
    formulas.append(
        OrList([
            AndList([TellTruth(john), Not(TellTruth(susan)), Not(TellTruth(mark)), Not(TellTruth(nicole))]),
            AndList([Not(TellTruth(john)), TellTruth(susan), Not(TellTruth(mark)), Not(TellTruth(nicole))]),
            AndList([Not(TellTruth(john)), Not(TellTruth(susan)), TellTruth(mark), Not(TellTruth(nicole))]),
            AndList([Not(TellTruth(john)), Not(TellTruth(susan)), Not(TellTruth(mark)), TellTruth(nicole)])
        ])
    )

    # Fact 5: Exactly one person crashed the server
    formulas.append(
        OrList([
            AndList([CrashedServer(john), Not(CrashedServer(susan)), Not(CrashedServer(mark)), Not(CrashedServer(nicole))]),
            AndList([Not(CrashedServer(john)), CrashedServer(susan), Not(CrashedServer(mark)), Not(CrashedServer(nicole))]),
            AndList([Not(CrashedServer(john)), Not(CrashedServer(susan)), CrashedServer(mark), Not(CrashedServer(nicole))]),
            AndList([Not(CrashedServer(john)), Not(CrashedServer(susan)), Not(CrashedServer(mark)), CrashedServer(nicole)])
        ])
    )
    
    # raise NotImplementedError()
    query = CrashedServer('$x')
    return (formulas, query)

In [262]:
# test case
predForms, predQuery = liar()
assert len(predForms) == 6

In [263]:
# test case
predForms, predQuery = liar()
formula_id = 0
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 2 models
Example model: {'CrashedServer(john)'}


In [264]:
# test case
predForms, predQuery = liar()
formula_id = 1
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 2 models
Example model: {'TellTruth(susan)', 'CrashedServer(nicole)'}


In [265]:
# test case
predForms, predQuery = liar()
formula_id = 2
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 2 models
Example model: set()


In [266]:
# test case
predForms, predQuery = liar()
formula_id = 3
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 2 models
Example model: {'TellTruth(susan)'}


In [267]:
# test case
predForms, predQuery = liar()
formula_id = 4
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 4 models
Example model: {'TellTruth(nicole)'}


In [268]:
# test case
predForms, predQuery = liar()
formula_id = 5
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 4 models
Example model: {'CrashedServer(john)'}


In [269]:
# test case
predForms, predQuery = liar()
assert checkFormula('3a-all', AndList(predForms)) == True

You matched the 1 models
Example model: {'CrashedServer(john)', 'TellTruth(nicole)'}


Hãy chạy cell dưới đây để xem bạn đã giải được bài toán đặt ra chưa nhé :) 

In [270]:
# Run this cell to solve the puzzle and find out who was the liar!
try:
    predForms, predQuery = liar()
    kb = createModelCheckingKB()

    filename = os.path.join('models', '3a-all.pklz')
    objects, targetModels = pickle.load(gzip.open(filename))
    for obj in objects:
        kb.tell(Atom('Object', obj))

    for predForm in predForms:
        response = kb.tell(predForm)

    response = kb.ask(predQuery)
    showKBResponse(response)
except:
    print('You failed!')

Yes: ['john']
Maybe: []
No: ['susan', 'nicole', 'mark']


## Câu 4: Hợp giải trên logic mệnh đề (5đ)

**Phần này các bạn tự thiết kế và code bằng các script python, không viết vào notebook này.**

Cho cơ sở tri thức **(KB)** và một câu $\alpha$, cả hai đều được biểu diễn bằng logic mệnh đề và được chuẩn hóa về dạng hội chuẩn CNF. Xác định KB entails $\alpha$ (KB $\vDash \alpha$) bằng hợp giải.

Gợi ý tham khảo: Sách Artificial Intelligence: A Modern Approach, Third Edition, Chương 7, Hình 7.12, hàm PL-RESOLUTION

- Báo cáo ít nhất 5 kịch bản kiểm thử để thể hiện chương trình chạy hợp lệ (và do đó kịch bản không nên quá đơn giản). 
- Đánh giá ngắn gọn về ưu điểm và khuyết điểm của giải thuật hợp giải trên logic mệnh đề, cũng như đề xuất giải pháp để khắc phục vấn dề.

**a. Đặc tả dữ liệu đầu vào và đầu ra**

- **Dữ liệu đầu vào**: **KB** và $\alpha$ theo dạng chuẩn CNF được lưu trong tập tin **input.txt**. Tập tin có định dạng quy ước như sau:
    - Dòng đầu tiên chứa câu $\alpha$
    - Dòng thứ hai chứa số nguyên N – số mệnh đề có trong KB
    - N dòng tiếp theo biểu diễn các mệnh đề trong KB, mỗi mệnh đề trên một dòng

Literal dương được biểu diễn bằng ký tự đơn viết hoa (A-Z). Literal âm là literal dương có dấu trừ (‘-‘) ngay trước ký tự

Từ khóa OR nối các literal nối với nhau. Có thể có một hay nhiều khoảng trắng giữa các literal và từ khóa

- **Dữ liệu đầu ra**: Tập hợp mệnh đề được phát sinh trong quá trình hợp giải và câu kết luận được lưu trong tập tin **output.txt**. Tập tin có định dạng quy ước như sau:
    - Dòng đầu tiên chứa số nguyên M1 – số mệnh đề được phát sinh trong vòng lặp đầu tiên. M1 dòng tiếp theo biểu diễn các mệnh đề được phát sinh trong vòng lặp đầu tiên (kể cả mệnh đề rỗng), mỗi mệnh đề trên một dòng. Mệnh đề rỗng được biểu diễn bằng chuỗi “{}”
    - Các vòng lặp tiếp theo (lần lượt có $M_2$, $M_3$,…, $M_n$ mệnh đề) được biểu diễn tương tự như trên
    - Dòng cuối cùng trình bày câu kết luận, tức là trả lời câu hỏi “KB entails $\alpha$?”. In YES nếu KB entails $\alpha$. Ngược lại, in NO.
    - Bỏ qua các mệnh đề trùng (xuất hiện trong cùng vòng lặp hay, KB ban đầu hay những vòng lặp trước đó).

- Hàm main phải thực hiện những thao tác cơ bản như sau:
    - Đọc dữ liệu đầu vào và lưu trong cấu trúc dữ liệu thích hợp
    - Gọi hàm PL-Resolution để thực thi giải thuật hợp giải
    - Ghi dữ liệu đầu ra vào tập tin đầu ra theo định dạng hợp lệ


- Lưu ý ngữ nghĩa của giá trị trả về true và false trong hàm PL-RESOLUTION. Đừng quên rằng khi hợp giải thì ta cần phủ định của câu $\alpha$.

- <font color='red'>Các literal trong cùng mệnh đề (đối với cả dữ liệu đầu vào và đầu ra) được xếp theo thứ tự chữ cái </font>
- <font color='red'>Kiểm tra điều kiện suy dẫn ở cuối mỗi vòng lặp, tức là khi đã phát sinh hết câu mới từ KB hiện hành, chứ không kiểm tra sau mỗi lần phát sinh một câu </font>
- <font color='red'>Các mệnh đề dạng</font> $A \vee B \vee -B$ <font color='red'>có chân trị</font> $True$ <font color='red'>vì tương đương với</font> $A \vee True$. <font color='red'>Những mệnh đề như thế này vô ích cho việc suy dẫn và do đó có thể bỏ đi </font>

Dữ liệu đầu vào được giả sử hợp lệ, các bạn không cần kiểm tra điều này.

Ví dụ với KB và câu $\alpha$ trong tập tin **input.txt**

| Input.txt | Output.txt | Ghi chú |
| --- | --- | --- |
| -A | 3 |  |
| 4 | -A | (-A OR B) hợp giải với (-B)|
| -A OR B | B | (-A OR B) hợp giải với (negative of -A) |
| B OR -C | -C | (-C OR B) hợp giải với (-B) |
| A OR -B OR C | 4 |  |
| -B | -B OR C | (A OR C OR -B) hợp giải với (-A) |
|  | A OR C | (A OR C OR -B) hợp giải với (B) |
|  | A OR -B| (A OR C OR -B) hợp giải với (-C) |
|  | {}| (-B) hợp giải với (B) |
|  | **YES** | KB entails $\alpha$ vì tồn tại mệnh đề rỗng trong KB |


Một ví dụ khác với cùng KB nhưng khác câu $\alpha$

| Input.txt | Output.txt | Ghi chú |
| --- | --- | --- |
| A | 2 | KB KHÔNG entail $\alpha$ vì không phát sinh được mệnh đề mới và không tìm thấy mệnh đề rỗng|
| 4 | -C |  |
| -A OR B | -B OR C |  |
| -C OR B | 2 |  |
| A OR C OR -B | A OR -B |  |
| -B | -A OR C |  |
|  | 1 |  |
|  | A OR -C |  |
|  | **0** |  |
|  | NO |  |







**b. Tiêu chí đánh giá**

| **STT** | **Đặc tả tiêu chí** | **Điểm** |
| --- | --- | --- |
| 1 | Đọc dữ liệu đầu vào và lưu trong cấu trúc dữ liệu phù hợp | 0.5 |
| 2 | Cài đặt giải thuật hợp giải trên logic mệnh đề  | 1 |
| 3 | Các bước suy diễn phát sinh đủ mệnh đề và kết luận đúng | 2.5 |
| 4 | Tuân thủ mô tả định dạng của đề bài | 0.5 |
| 5 | Báo cáo test case và đánh giá | 0.5 |